# AstraForge Toolkit — Local API smoke test

Quick sanity checks for the published package against a locally running AstraForge API, plus the same quick-start snippets from the README.

Prereqs:
- Backend running at `http://localhost:8000` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install -e ../astraforge-python-package` or from PyPI)
- For the DeepAgent example, set `OPENAI_API_KEY` (or configure your preferred model) so LangChain can call a chat model.


In [ ]:
# Install the local package in editable mode (run from the examples/ folder)
%pip install -e ..

In [ ]:
%pip install python-dotenv

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

BASE_URL = "http://localhost:8000/api"
API_KEY = os.getenv("ASTRA_FORGE_API_KEY")

### Create a sandbox-backed DeepAgent (README quick start)

Same as the README example: configure a SandboxBackend, pick a chat model, and build a DeepAgent that can use sandbox tools. Reuses one sandbox session and thread across multiple invocations.


In [ ]:
# Install a model provider helper for the example (skip if already installed)
%pip install langchain-openai

In [ ]:
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI
from astraforge_toolkit import (
    DeepAgentClient,
    SandboxBackend,
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
)

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
conv = client.create_conversation()
THREAD_ID = conv.conversation_id
SANDBOX_SESSION_ID = conv.sandbox_session_id
print(f"Using sandbox session {SANDBOX_SESSION_ID} for multiple calls")

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        session_id=SANDBOX_SESSION_ID,
        # optional: session_params={"image": "astraforge/codex-cli:latest"},
    )

model = ChatOpenAI(model="gpt-4o-mini")  # or any chat model you prefer
tools = [
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
]

deep_agent = create_deep_agent(
    model=model,
    backend=backend_factory,
    tools=tools,
)

RUN_CONFIG = {
    "thread_id": THREAD_ID,
    "configurable": {"sandbox_session_id": SANDBOX_SESSION_ID},
}

Using sandbox session a7743c4b-aa21-45ab-a199-e462ffc5647d for multiple calls


In [5]:
deep_agent.invoke(
    {"messages": [{"role": "user", "content": "List files in /workspace"}]},
    config=RUN_CONFIG,
)

# Reuse RUN_CONFIG for follow-up calls to keep the same sandbox session and thread ID.

{'messages': [HumanMessage(content='List files in /workspace', additional_kwargs={}, response_metadata={}, id='3b7731d6-1dc4-4f65-bb3b-ac7f723cbcab'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 4505, 'total_tokens': 4520, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4480}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_67cf3fed12', 'id': 'chatcmpl-CheGx2frlTZlAF6beEx4JLZvmem1M', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3c589a20-7a1b-4586-a05c-336240e8c640-0', tool_calls=[{'name': 'ls', 'args': {'path': '/workspace'}, 'id': 'call_Jhk89EX4xvn4eLtYUDyXBtOX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 4505, 'output_tokens': 15, 'total_to